In [3]:
import os
import sys
import pprint 
import random
from collections import defaultdict

from bs4 import BeautifulSoup
import nltk

from IPython.display import display, HTML

sys.path.append(os.path.abspath('../../WKP-python-toolkit'))
import wekeypedia

In [4]:
p = wekeypedia.WikipediaPage("Love")

revisions_list = p.get_revisions_list()

In [5]:
diff_test = p.get_diff(revisions_list[10]["revid"])

display(HTML(diff_test))

In [164]:
def normalize(word):
  # old = word
  word = word.lower()
  word = stemmer.stem_word(word)
  word = lemmatizer.lemmatize(word)
  # print word
  return word

lemmatizer = nltk.WordNetLemmatizer()
stemmer = nltk.stem.porter.PorterStemmer()

def extract(sentence):
    diff = { "added": [],
             "deleted" : [] }

    d = BeautifulSoup(sentence, 'html.parser')

    # FIXME: there is an issue with diff including simple modification
    # within paragraph without deletion or addition
    tr = d.find_all("tr")
    
    # check additions of block
    added = []
    
    # added = d.find_all("td", "diff-addedline")
    added = [ t.find("td", "diff-addedline") for t in tr if len(t.find_all("ins")) == 0 and len(t.find_all("td", "diff-empty")) > 0 ]
    added = [ x for x in added if x != None ]
    added.extend(d.find_all("ins"))
    
    diff["added"] = [ tag.get_text() for tag in added ]

    # check deletions of block
    deleted = []
#     deleted = d.find_all("td", "diff-deleted")
#     deleted = [ t for t in deleted if len(t.find_all("del")) == 0 ]
    deleted = [ t.find("td", "diff-deletedline") for t in tr if len(t.find_all("del")) == 0 and len(t.find_all("td", "diff-empty")) > 0 ]
    deleted = [ x for x in deleted if x != None ]
    deleted.extend(d.find_all("del"))

    diff["deleted"] = [ tag.get_text() for tag in deleted ]

    return diff

def count_stems(diff, inflections):
    stems = {}
    stems["added"] = defaultdict(int)
    stems["deleted"] = defaultdict(int)

    for which in diff.keys():
        for sentence in diff[which]:
            for word in nltk.word_tokenize(sentence):
                old = word
                word = normalize(word)
                if not(word in ignore_list):
                    stems[which][word] += 1

                    # keeping track of inflection usages
                    inflections[word].setdefault(old,0)
                    inflections[word][old] += 1

    return stems

#   # bug with Ethics#462124891
#   if d == False:
#     return ({},{}, { "added": [], "deleted": [] })


def diff_terms_counts(diff):
  for txt in map(lambda x: x.get_text(), del_tags):
    count(de, txt, "deleted")

  #print("%s %s|%s" % (revid, "+"*len(ad), "-"*len(de)))

  return (ad.items(),de.items(), { "added":inflections["added"], "deleted":inflections["deleted"] })

In [173]:
inflections = defaultdict(dict)

ignore_list = "{}()[]<>./,;\"':!?&#="

diff = p.get_diff(random.choice(revisions_list)["revid"])

display(HTML(diff))

diff = extract(diff)

for minus in diff["deleted"]:
    print "- %s" % (minus)

for plus in diff["added"]:
    print "+ %s" % (plus)

print count_stems(diff, inflections)

for stem, i in inflections.iteritems():
    print "[%s] %s" % (stem, ", ".join(map(lambda x: "%s (%s)" % (x[0], x[1]), i.items())))

- {{otheruses}}{{Close Relationships}}
- '''Love''' is a profound [[feeling]] of tender [[affection]] for or intense [[interpersonal attraction|attraction]] to another. It is considered a deep, [[ineffability|ineffable]] feeling shared in passionate or [[intimacy|intimate]] [[interpersonal relationship]]s. However, in different contexts, the word ''love'' has a variety of related but distinct meanings: in addition to [[romantic love]], which is characterized by a mix of [[emotion]]al and [[human sexuality|sexual]] desire, other forms include [[platonic love]], [[love (religious views)|religious love]], [[familial love]], and the more casual application of the term to anyone or anything that one considers strongly pleasurable, enjoyable, or desirable, including activities and foods. This diverse range of meanings in a single word is commonly contrasted with the plurality of [[Greek words for love|Greek words for ''love'']], reflecting the word's versatility and complexity.
- 
- Although